In [1]:
# Test if basic Python works
print("Hello World")
import sys
print(f"Python version: {sys.version}")

Hello World
Python version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [ ]:
!pip install sounddevice transformers librosa matplotlib noisereduce


In [2]:
!pip install -q datasets transformers torchaudio librosa noisereduce


^C


In [ ]:
import sounddevice as sd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch
import noisereduce as nr
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Recording parameters
DURATION = 5          # seconds
SAMPLE_RATE = 16000   # wav2vec2 expects 16kHz

print("🎙️ Recording... Speak now!")
audio = sd.rec(int(DURATION * SAMPLE_RATE),
               samplerate=SAMPLE_RATE,
               channels=1,
               dtype='float32')
sd.wait()
print("✅ Recording finished")

speech = audio.flatten()


In [ ]:
plt.figure(figsize=(10,4))
S = librosa.feature.melspectrogram(y=speech, sr=SAMPLE_RATE)
S_db = librosa.power_to_db(S, ref=np.max)

librosa.display.specshow(S_db, sr=SAMPLE_RATE, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title("Live Audio Spectrogram")
plt.tight_layout()
plt.show()


In [ ]:
speech_clean = nr.reduce_noise(y=speech, sr=SAMPLE_RATE)


In [ ]:
processor = Wav2Vec2Processor.from_pretrained(
    "facebook/wav2vec2-large-960h"
)

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h"
)


In [ ]:
input_values = processor(
    speech_clean,
    sampling_rate=SAMPLE_RATE,
    return_tensors="pt",
    padding=True
).input_values

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

print("\n📝 TRANSCRIPTION:")
print(transcription)


In [ ]:
!pip install pydub
!apt-get install ffmpeg

!pip install -q jiwer
!pip install -q pyctcdecode
!pip install -q kenlm
!pip install -q textblob
!python -m textblob.download_corpora
